In [14]:
##clache

import os
import cv2

image_folder = '1'
processed_folder = 'augmented_images1'

if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    
    if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        image = cv2.imread(image_path)
        if image is None:
            continue  
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        processed_image = clahe.apply(gray_image)
        
        processed_image_path = os.path.join(processed_folder, image_name)
        cv2.imwrite(processed_image_path, processed_image)


In [15]:

##canne 

import os
import cv2

image_folder = '1'  
processed_folder = 'augmented_images2'

if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    
    if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        image = cv2.imread(image_path)
        if image is None:
            continue  
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        processed_image = clahe.apply(gray_image)
        
        processed_image_path = os.path.join(processed_folder, image_name)
        cv2.imwrite(processed_image_path, processed_image)


In [16]:
import numpy as np
import os
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

input_folder_path = '1' 
output_dir = 'augmented_images3'
number_of_augmented_images = 5  
os.makedirs(output_dir, exist_ok=True)

datagen = ImageDataGenerator(
    rotation_range=10, 
    width_shift_range=0.05,  
    height_shift_range=0.05,  
    horizontal_flip=True,  
    fill_mode='nearest',  
    zoom_range=[0.95, 1.05] 
)

for image_path in glob.glob(os.path.join(input_folder_path, '*.jpg')):
    img = load_img(image_path)  
    x = img_to_array(img) 
    x = x.reshape((1,) + x.shape) 

    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=output_dir, save_prefix='aug', save_format='jpeg'):
        i += 1
        if i >= number_of_augmented_images:
            break 

print(f"Generated {number_of_augmented_images} augmented images per original image in '{output_dir}' directory.")


Generated 5 augmented images per original image in 'augmented_images3' directory.


In [4]:
import albumentations as A
import cv2
import os
from tqdm import tqdm  

transform = A.Compose([
    A.Rotate(limit=20, p=0.9),
    A.HorizontalFlip(p=0.5),
    A.RandomCrop(width=400, height=400, p=0.1),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
    A.GaussNoise(var_limit=(10.0, 30.0), p=0.5),
    A.Blur(blur_limit=3, p=0.5),
])

source_folder = 'Beds'
save_folder = 'Beds_Augmented'

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

for filename in tqdm(os.listdir(source_folder), desc='Augmenting Images'):
    image_path = os.path.join(source_folder, filename)
    
    image = cv2.imread(image_path)
    

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for i in range(6):
        augmented = transform(image=image)
        augmented_image = augmented['image']
        
        save_path = os.path.join(save_folder, f"augmented_{os.path.splitext(filename)[0]}_{i}.jpg")
        
        cv2.imwrite(save_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

print("All augmented images have been saved.")


Augmenting Images: 100%|██████████| 3421/3421 [24:56<00:00,  2.29it/s] 

All augmented images have been saved.


In [3]:
from PIL import Image
import os

def delete_small_photos(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        with Image.open(file_path) as img:
            if img.width < 700 or img.height < 700:
                os.remove(file_path)
        

directory_path = 'Beds'
delete_small_photos(directory_path)


In [1]:
import torch
import kornia as K
import cv2
import os
from torchvision.transforms.functional import to_pil_image

def convert_image_to_sketch_kornia(image_path, output_path, blur_size=5, threshold1=100, threshold2=200):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img_tensor = K.image_to_tensor(img, keepdim=False).float() / 255.0
    blur_kernel_size = (blur_size, blur_size)
    if blur_size % 2 == 0:
        blur_kernel_size = (blur_size + 1, blur_size + 1)
    img_blurred = K.filters.gaussian_blur2d(img_tensor, blur_kernel_size, (blur_size, blur_size))

    gradients = K.filters.spatial_gradient(img_blurred, order=1, mode='sobel')
    edges_x = gradients[:, 0, :, :]
    edges_y = gradients[:, 1, :, :]
    edges = torch.sqrt(edges_x ** 2 + edges_y ** 2)
    edges = (edges - edges.min()) / (edges.max() - edges.min())  
    edges = (edges > threshold1 / 255.0).float()  
    edges_pil = to_pil_image(edges.squeeze())

    if edges_pil.mode != 'L':
        edges_pil = edges_pil.convert('L')

    edges_pil.save(output_path, format='JPEG') 
def process_folder_kornia(input_folder, output_folder, blur_size=5, threshold1=100, threshold2=200):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f'sketch_{filename}')
            
            convert_image_to_sketch_kornia(file_path, output_path, blur_size, threshold1, threshold2)
            print(f"Processed and saved: {output_path}")

input_folder = '1'  
output_folder = 'augmented_images5' 

blur_size = 5 
threshold1 = 100
threshold2 = 200

process_folder_kornia(input_folder, output_folder, blur_size, threshold1, threshold2)
